In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

In [2]:
!pip install -qU ragas

In [3]:
!pip install -qU qdrant-client pymupdf pandas

In [1]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "data/meta10k.pdf",
)

documents = loader.load()

In [3]:
documents[0].metadata

{'source': 'data/meta10k.pdf',
 'file_path': 'data/meta10k.pdf',
 'page': 0,
 'total_pages': 147,
 'format': 'PDF 1.4',
 'title': '0001326801-24-000012',
 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group',
 'subject': 'Form 10-K filed on 2024-02-02 for the period ending 2023-12-31',
 'keywords': '0001326801-24-000012; ; 10-K',
 'creator': 'EDGAR Filing HTML Converter',
 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
 'creationDate': "D:20240202060356-05'00'",
 'modDate': "D:20240202060413-05'00'",
 'trapped': '',
 'encryption': 'Standard V2 R3 128-bit RC4'}

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Note: we can play around with chunksize here....

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)
len(documents)

636

In [5]:
from langchain_openai import OpenAIEmbeddings

# we can also play around with the embeddings here... 
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

In [6]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="Meta10k",
)

In [7]:
retriever = qdrant_vector_store.as_retriever()

In [8]:
retrieved_documents = retriever.invoke("Who are Meta's 'Directors' (i.e., members of the Board of Directors)?")
for doc in retrieved_documents:
  print(doc.page_content)

workforce.
We want our products to work for people around the world and we need to grow and keep the best talent in order to do that. We also remain committed
to having a skilled, inclusive and diverse workforce because we believe cognitive diversity fuels innovation. To aid in this effort, we have taken steps to reduce
bias from our hiring processes and performance management systems, as well as offering learning and development courses for our employees.
Corporate Information
We were incorporated in Delaware in July 2004. We completed our initial public offering in May 2012 and our Class A common stock is currently listed
on the Nasdaq Global Select Market under the symbol "META." Our principal executive offices are located at 1 Meta Way, Menlo Park, California 94025, and
our telephone number is (650) 543-4800.
Meta, the Meta logo, Meta Quest, Meta Horizon, Facebook, FB, Instagram, Oculus, WhatsApp, Reels, and our other registered or common law
Robert M. Kimmitt
/s/ Sheryl K. Sandber

In [13]:
retrieved_documents = retriever.invoke("What was the total value of 'Cash and cash equivalents' as of December 31, 2023?")
for doc in retrieved_documents:
  print(doc.page_content)
  print('-----------------')

•
Capital expenditures, including principal payments on finance leases, were $28.10 billion for the year ended December 31, 2023.
•
Effective tax rate was 17.6% for the year ended December 31, 2023.
•
Cash, cash equivalents, and marketable securities were $65.40 billion as of December 31, 2023.
•
Long-term debt was $18.39 billion as of December 31, 2023.
•
Headcount was 67,317 as of December 31, 2023, a decrease of 22% year-over-year.
Dividend
Prior to 2024, we had never declared or paid any cash dividend on our common stock. On February 1, 2024 we announced the initiation of our first ever
cash dividend program. This cash dividend of $0.50 per share of Class A common stock and Class B common stock (together, the “common stock”) is
equivalent to $2.00 per share on an annual basis. The first cash dividend will be paid on March 26, 2024 to all holders of record of common stock at the close
of business on February 22, 2024.
60
-----------------
Table of Contents
 
 
Fair Value Measurement

In [14]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [15]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [16]:
question = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

$41.862 billion


In [17]:
for c in result["context"]:
  print(c.page_content)
  print('-----------------')


•
Capital expenditures, including principal payments on finance leases, were $28.10 billion for the year ended December 31, 2023.
•
Effective tax rate was 17.6% for the year ended December 31, 2023.
•
Cash, cash equivalents, and marketable securities were $65.40 billion as of December 31, 2023.
•
Long-term debt was $18.39 billion as of December 31, 2023.
•
Headcount was 67,317 as of December 31, 2023, a decrease of 22% year-over-year.
Dividend
Prior to 2024, we had never declared or paid any cash dividend on our common stock. On February 1, 2024 we announced the initiation of our first ever
cash dividend program. This cash dividend of $0.50 per share of Class A common stock and Class B common stock (together, the “common stock”) is
equivalent to $2.00 per share on an annual basis. The first cash dividend will be paid on March 26, 2024 to all holders of record of common stock at the close
of business on February 22, 2024.
60
-----------------
Table of Contents
 
 
Fair Value Measurement

In [18]:
question = "Who are Meta's 'Directors' (i.e., members of the Board of Directors)?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Robert M. Kimmitt, Sheryl K. Sandberg, Tracey T. Travis, Tony Xu


In [19]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. 
When answer about financial information, look for content only in table form.
If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [20]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [29]:
question = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

$41.862 billion


In [43]:
for c in result["context"]:
  print(c.page_content)
  print('-----------------')

holdings are in diversified highly rated securities. Cash, cash equivalents, and marketable securities were $65.40 billion as of December 31, 2023, an increase
of $24.67 billion from December 31, 2022. The increase was mostly due to $71.11 billion of cash generated from operations, and $8.46 billion of net proceeds
from the issuance of fixed-rate senior unsecured notes (the Notes) in May 2023. These increases were partially offset by $28.10 billion for capital expenditures,
-----------------
261 
— 
261 
— 
Corporate debt securities
220 
— 
220 
— 
Total cash and cash equivalents
41,862 
35,116 
481 
— 
Marketable securities:
U.S. government securities
8,439 
8,439 
— 
— 
U.S. government agency securities
3,498 
3,498 
— 
— 
Corporate debt securities
11,604 
— 
11,604 
— 
Total marketable securities
23,541 
11,937 
11,604 
— 
Restricted cash equivalents
857 
857 
— 
— 
Other assets
101 
— 
— 
101 
Total
$
66,361 
$
47,910 
$
12,085 
$
101 
107
-----------------
Table of Contents
 
 
Fa

In [27]:
question = "Who are Meta's 'Directors' (i.e., members of the Board of Directors) on the signature page?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

I don't know.


In [28]:
for c in result["context"]:
  print(c.page_content)
  print('-----------------')

Table of Contents
SIGNATURES
Pursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this Annual Report on Form
10-K to be signed on its behalf by the undersigned, thereunto duly authorized, in the City of Menlo Park, State of California, on this 1st day of February 2024.
META PLATFORMS, INC.
Date:
February 1, 2024
/s/ Susan Li 
Susan Li
Chief Financial Officer
 
131
-----------------
or his or her substitute or substitutes, may lawfully do or cause to be done by virtue hereof.
Pursuant to the requirements of the Securities Exchange Act of 1934, this Annual Report on Form 10-K has been signed by the following persons on
behalf of the Registrant and in the capacities and on the dates indicated: 
Signature
Title
Date
/s/ Mark Zuckerberg
Board Chair and Chief Executive Officer
(Principal Executive Officer)
February 1, 2024
Mark Zuckerberg
/s/ Susan Li
Chief Financial Officer
(Principal Financial Officer)
February 1, 2024
S